#### Summary

The idea here is to read betas and corresponding columns from the given csv file and generate predictions on test data using the same.

In [1]:
import pandas as pd
import numpy as np
import time as time

In [2]:
INPUT_DIR = '../input/'

In [3]:
ts = time.time()
test_data = pd.read_csv(INPUT_DIR + 'test.csv')
time.time() - ts

72.7997829914093

In [6]:
def write_lasso_model_predictions_to_file(test_data, lasso_model_file, output_predictions_file):
    lasso_model = pd.read_csv(lasso_model_file)
    
    # Rename the column name, which is obtained from the R output file.
    lasso_model.rename(columns={'Unnamed: 0':'column_name', 'x': 'coefficient'}, inplace=True)
    
    # R normalizes the column names by adding X if it starts with a number. Make sure that we take care of it.
    lasso_model['column_name'] = lasso_model['column_name'].apply(lambda x : x.lstrip('X'))
    
    # Gets the non zero coefficients to dictionary for easier manipulations later.
    col_to_coef_dict = dict(zip(lasso_model.column_name, lasso_model.coefficient))
    
    # Remove the ID as we will not be using it for predictions.
    test_data_without_ID = test_data.drop(columns=['ID'])
    
    # Populates the model coefficients to a matrix for ease in multiplications later.
    columns_coef_matrix = np.zeros((len(test_data_without_ID.columns), 1))
    
    i = 0
    for col in test_data_without_ID.columns:
        columns_coef_matrix[i, 0] = col_to_coef_dict.get(col, 0)
        i += 1    
    
    print(np.sum(columns_coef_matrix > 0))
    test_data_without_ID_matrix = test_data_without_ID.to_numpy()
    
    
    # Gets predictions by matrix multiplication.
    predictions = test_data_without_ID_matrix @ columns_coef_matrix
    
    test_data['target'] = predictions
    test_data[['ID', 'target']].to_csv(output_predictions_file, index=False)
    

In [7]:
ts = time.time()
write_lasso_model_predictions_to_file(test_data, 
                                      'minimum_cv_lasso_model_columns_and_coefficients.csv',
                                      'minimum_cv_lasso_model_predictions.csv')
time.time() - ts

20


3.7546470165252686

In [8]:
ts = time.time()
write_lasso_model_predictions_to_file(test_data, 
                                      'one_se_cv_lasso_model_columns_and_coefficients.csv',
                                      'one_se_cv_lasso_model_predictions.csv')
time.time() - ts

48


1.4280719757080078